# Which correlations do we want

simulate destination mode choice model:
 1) MC
 2) EET fixed error per level
 3) EET only fixed when identical

Does it make a difference in analysis?

In [32]:
import pandas as pd
import numpy as np

from numpy.random import default_rng
from copy import deepcopy

In [33]:
rng = default_rng(999)

### Zones
num_zones = 10
destination = pd.DataFrame(
    data=zip(np.arange(0, num_zones), rng.normal(1000, 10, size=num_zones), strict=True),
    columns=['zone', 'attrs'],
)

### People - assume each person makes one trip from random origin
num_ppl = 10000
vars=['id', 'income', 'orig']
ppl = pd.DataFrame(
    data=zip(
        np.arange(0, num_ppl),
        rng.normal(50, 20, size=(num_ppl)).clip(2, 150),
        rng.choice(np.arange(0, num_zones), num_ppl),
        strict=True,
    ),
    columns=vars,
)


### Modes
num_modes = 3
mode_idx_to_str = {0: 'car', 1: 'pt', 2: 'walk'}
modes = np.array(['car', 'pt', 'walk'])
mode_asc = {'car': 2.5, 'pt': 0.0, 'walk': 10.0}

### Cost and time by mode
skims = {}
skims['time'] = {}
skims['time']['car'] = rng.normal(20, 20, size=(num_zones, num_zones)).clip(4, 60)
skims['time']['pt'] = rng.normal(40, 10, size=(num_zones, num_zones)).clip(4, 80)
skims['time']['walk'] = 3.0 * skims['time']['car'].copy()

skims['cost'] = {}
skims['cost']['car'] = skims['time']['car'].copy() / 4.0
skims['cost']['pt'] = 2.0 * np.ones_like(skims['time']['pt'])
skims['cost']['walk'] = np.zeros_like(skims['time']['walk'])


# scenario: decrease pt travel time to (and within zone) 0 by 70%
scenario_skims = deepcopy(skims)
#scenario_skims['time']['pt'][0,:] *= 0.3
scenario_skims['time']['pt'][:,0] *= 0.3

In [34]:
def logsum(utilities, nest_scale=1.0):
    scaled_utils = utilities / nest_scale
    max_util = np.max(scaled_utils, axis=1)
    return max_util + np.log(np.sum(np.exp(scaled_utils - max_util.reshape((-1,1))), axis=1))

def inverse_ev1_cdf(x, location=0.0, scale=1.0):
    return location - scale * np.log(-np.log(x))

### Utility function parameters
beta_time = {}
beta_time['car'] = -0.06
beta_time['pt'] = -0.03
beta_time['walk'] = -0.2
beta_cost = -0.01

def mode_utility(mode, income, o, d, skims):
    return (
        beta_time[mode] * skims['time'][mode][o,d]
        + beta_cost * skims['cost'][mode][o,d] / income
        + mode_asc[mode]
    )

def mode_logsum(o: int, d: np.array, income: float, skims):
    mode_utils = np.array([mode_utility(m_, income, o, d, skims) for m_ in modes]).T

    return logsum(mode_utils, 1.0)

# mode_logsum(0, np.array([0, 1, 2]), 100)

In [35]:
#inverse_ev1_cdf(default_rng(seed=np.random.SeedSequence([777, 333])).random(100))

In [36]:
beta_logsum = -0.1

dest_altneratives = destination.zone.values
num_dests = dest_altneratives.shape[0]
dest_attrs = np.log(destination['attrs'].values)
ppl['dest_utils'] = ppl.apply(
    lambda x: (
        beta_logsum * mode_logsum(int(x['orig']), dest_altneratives, x['income'], skims)
        + dest_attrs
    ),
    axis=1,
)
ppl['dest_utils_scen'] = ppl.apply(
    lambda x: (
        beta_logsum * mode_logsum(int(x['orig']), dest_altneratives, x['income'], scenario_skims)
        + dest_attrs
    ),
    axis=1,
)

In [37]:
### EET
def destination_choice_model(ppl, seed=777, util_column_name="dest_utils"):
    dest_choice = ppl.apply(
        lambda x: np.argmax(
            x[util_column_name] + inverse_ev1_cdf(
                default_rng(seed=np.random.SeedSequence([seed, int(x['id']), 100])).random(size=num_dests)
            )
        )
        , axis=1
    )
    return dest_choice

def mode_choice_model(ppl, skims, dest_col_name='dest', seed=777, od_seed_contrib=False):
    """"od_seed_contrib: if True, seed depedns on destination"""
    def seed_o_d(d):
        if od_seed_contrib:
            return d
        return 0

    mode_choice = ppl.apply(
        lambda x: np.argmax(
            [mode_utility(m_, x['income'], int(x['orig']), int(x[dest_col_name]), skims) for m_ in modes]
            + inverse_ev1_cdf(default_rng(seed=np.random.SeedSequence(
                [seed, int(x['id']), 200, seed_o_d(int(x[dest_col_name]))]
            )).random(size=num_modes))
        ),
        axis=1,
    )
    #mode_choice = mode_utils.apply(np.argmax)
    return mode_choice.map(mode_idx_to_str)

### MC
def destination_choice_model_mc(ppl, seed=777, util_column_name="dest_utils"):
    dest_choice = ppl.apply(
        lambda x: default_rng(
            seed=np.random.SeedSequence([seed, int(x['id']), 100])
        ).choice(dest_altneratives, p=np.exp(x[util_column_name]) / np.sum(np.exp(x[util_column_name])), size=1)[0],
        axis=1,
    )
    return dest_choice

def mode_choice_model_mc(ppl, skims, dest_col_name='dest', seed=777, od_seed_contrib=False):
    def seed_o_d(d):
        if od_seed_contrib:
            return d
        return 0

    ppl['mode_exp_utils'] = ppl.apply(
        lambda x: np.exp(
            [mode_utility(m_, x['income'], int(x['orig']), int(x[dest_col_name]), skims) for m_ in modes]
        ),
        axis=1,
    )
    mode_choice = ppl.apply(
        lambda x: default_rng(
            seed=np.random.SeedSequence([seed, int(x['id']), 200, seed_o_d(int(x[dest_col_name]))])
        ).choice(modes, p=x.mode_exp_utils / np.sum(x.mode_exp_utils), size=1)[0],
        axis=1,
    )
    ppl.drop(columns=['mode_exp_utils'], inplace=True)
    return mode_choice

In [38]:
#for i in range(0, 10):
#    ppl.dest_utils.apply(lambda x: (np.exp(x) / np.sum(np.exp(x)))[i]).hist(figsize=(3,2))

In [39]:
# 10k ppl: per seed about 3.5s total [w/o sedd, with 12.4]
for s_ in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    ppl[f'dest_{s_}'] = destination_choice_model(ppl, util_column_name='dest_utils', seed=s_)
    ppl[f'dest_scen_{s_}'] = destination_choice_model(ppl, util_column_name='dest_utils_scen', seed=s_)
    ppl[f'dest_mc_{s_}'] = destination_choice_model_mc(ppl, util_column_name='dest_utils', seed=s_)
    ppl[f'dest_mc_scen_{s_}'] = destination_choice_model_mc(ppl, util_column_name='dest_utils_scen', seed=s_)
    
    
    ppl[f'mode_choice_{s_}'] = mode_choice_model(ppl, skims, dest_col_name=f'dest_{s_}', seed=s_)
    ppl[f'mode_choice_scen_{s_}'] = mode_choice_model(ppl, scenario_skims, dest_col_name=f'dest_scen_{s_}', seed=s_)
    #ppl[f'mode_choice_seedd_{s_}'] = mode_choice_model(ppl, skims, dest_col_name=f'dest_{s_}', seed=s_, od_seed_contrib=True)
    #ppl[f'mode_choice_seedd_scen_{s_}'] = mode_choice_model(ppl, scenario_skims, dest_col_name=f'dest_scen_{s_}', seed=s_, od_seed_contrib=True)

    ppl[f'mode_choice_mc_{s_}'] = mode_choice_model_mc(ppl, skims, dest_col_name=f'dest_mc_{s_}', seed=s_)
    ppl[f'mode_choice_mc_scen_{s_}'] = mode_choice_model_mc(ppl, scenario_skims, dest_col_name=f'dest_mc_scen_{s_}', seed=s_)
    #ppl[f'mode_choice_mc_seedd_{s_}'] = mode_choice_model_mc(ppl, skims, dest_col_name=f'dest_mc_{s_}', seed=s_, od_seed_contrib=True)
    #ppl[f'mode_choice_mc_seedd_scen_{s_}'] = mode_choice_model_mc(ppl, scenario_skims, dest_col_name=f'dest_mc_scen_{s_}', seed=s_, od_seed_contrib=True)

In [40]:
def display_metric(metric, ppl, seed):
    return ( 
        ppl[f'{metric}_{seed}'].value_counts().to_frame(f'eet_base_{seed}').join(
        ppl[f'{metric}_mc_{seed}'].value_counts().to_frame(f'mc_base_{seed}'), how='outer').join(
        ppl[f'{metric}_scen_{seed}'].value_counts().to_frame(f'eet_scen_{seed}'), how='outer').join(
        ppl[f'{metric}_mc_scen_{seed}'].value_counts().to_frame(f'mc_scen_{seed}'), how='outer').fillna(0).astype(int)
    )
        # ppl[f'mode_choice_seedd_{seed}'].value_counts().to_frame(f'eet_base_seedd_{seed}'), how='outer').join(
        # ppl[f'mode_choice_mc_seedd_{seed}'].value_counts().to_frame(f'mc_base_seedd_{seed}'), how='outer').join(
        # ppl[f'mode_choice_seedd_scen_{seed}'].value_counts().to_frame(f'eet_scen_seedd_{seed}'), how='outer').join(
        # ppl[f'mode_choice_mc_seedd_scen_{seed}'].value_counts().to_frame(f'mc_scen_seedd_{seed}'), how='outer')\

In [41]:
#pd.concat([display_metric('dest', ppl, i) for i in range(0, 3)], axis=1)

In [42]:
pd.concat([display_metric('mode_choice', ppl, i) for i in range(0, 3)], axis=1)

,eet_base_0,mc_base_0,eet_scen_0,mc_scen_0,eet_base_1,mc_base_1,eet_scen_1,mc_scen_1,eet_base_2,mc_base_2,eet_scen_2,mc_scen_2
car,6191,6215,6048,6060,6170,6117,6013,5962,6163,6201,6000,6065
walk,2318,2312,2322,2324,2361,2390,2367,2405,2389,2373,2395,2380
pt,1491,1473,1630,1616,1469,1493,1620,1633,1448,1426,1605,1555


In [60]:
metric = "mode_choice"

dfs_ = []

for seed in range(0, 10):
    for seed_2 in range(0, 10):
        df = ppl[f'{metric}_{seed}'].value_counts().to_frame(f'eet_base_{seed}').join(
                ppl[f'{metric}_scen_{seed_2}'].value_counts().to_frame(f'eet_scen_{seed_2}'), how='outer').join(
                ppl[f'{metric}_mc_{seed}'].value_counts().to_frame(f'mc_base_{seed}'), how='outer').join(
                ppl[f'{metric}_mc_scen_{seed_2}'].value_counts().to_frame(f'mc_scen_{seed_2}'), how='outer'
            ).fillna(0).astype(int)

        df[f'diff_eet_{seed}_{seed_2}'] = df[f'eet_scen_{seed_2}'] - df[f'eet_base_{seed}']
        df[f'diff_mc_{seed}_{seed_2}'] = df[f'mc_scen_{seed_2}'] - df[f'mc_base_{seed}']
        dfs_.append(df[[f'diff_eet_{seed}_{seed_2}', f'diff_mc_{seed}_{seed_2}']])

diff_mc = pd.concat(dfs_, axis=1)

In [61]:
# check eet 

diff_mc

,diff_eet_0_0,diff_mc_0_0,diff_eet_0_1,diff_mc_0_1,diff_eet_0_2,diff_mc_0_2,diff_eet_0_3,diff_mc_0_3,diff_eet_0_4,diff_mc_0_4,...,diff_eet_9_5,diff_mc_9_5,diff_eet_9_6,diff_mc_9_6,diff_eet_9_7,diff_mc_9_7,diff_eet_9_8,diff_mc_9_8,diff_eet_9_9,diff_mc_9_9
car,-143,-155,-178,-253,-191,-150,-164,-264,-181,-179,...,-230,-112,-30,-174,-160,-153,-156,-205,-166,-136
walk,4,12,49,93,77,68,21,50,55,106,...,64,-85,-32,-44,-5,-29,-18,18,13,-16
pt,139,143,129,160,114,82,143,214,126,73,...,166,197,62,218,165,182,174,187,153,152


In [ ]:
# increase in pt trips to/from zone 0


# increase/decrease in other areas -> and why. NEST SWITCHING? should not switch away from 0, right?